In [1]:
import pandas as pd
import altair as alt


In [3]:
success_table= pd.read_csv('supp_fig4_input.tsv', sep='\t')
print(len(set(success_table['ID'])),'genes')
print(len(success_table), 'variants')

39 genes
60 variants


In [5]:
mapping = pd.read_csv('GS_ID_mapping.csv')

mapped= []
for i, row in success_table.iterrows():
    ID = row['ID']
    mapped_id = mapping[mapping['ID']==ID]['Dumb_ID'].item()
    mapped.append(mapped_id)
success_table['Dumb_ID'] = mapped

In [6]:
def filt_vs_unfilt_df(unfiltered_run_type, filtered_run_type, success_table, limit,domain):
    data = []
    to_plot = []
    big_change=[]
    plot3=[]
    lost=[]
    too_big = []

    successes = success_table[success_table['Variant_Level_noMOI_'+str(unfiltered_run_type)] == 'Variant_Present_noMOI']

    for i, patient_row in successes.iterrows():
        ID = patient_row['Dumb_ID']
        unfiltered_rank = patient_row['Variant_Level_noMOI_rank_' + unfiltered_run_type]
        filtered_rank = patient_row['Variant_Level_noMOI_rank_' + filtered_run_type]
        if filtered_rank == 'N/A':
            filtered_rank = 0
            print(ID, unfiltered_rank)
        difference = abs(unfiltered_rank - filtered_rank)
        data.append([ID, unfiltered_rank, difference, str(unfiltered_run_type)])
        data.append([ID, filtered_rank, difference, str(filtered_run_type)])

        if unfiltered_rank != filtered_rank:
            if unfiltered_rank == 0 or filtered_rank == 0:
                lost.append(ID)
            else:
                if unfiltered_rank > 1000:
                    too_big.append(ID)
                else:
                    if unfiltered_rank > limit:
                        big_change.append(ID)
                    else:
                        to_plot.append(ID)
                    # if difference >30:
                    #     to_plot.append(ID)
        if difference > 10:
            plot3.append(ID)

    df = pd.DataFrame(data, columns=['ID', 'Rank', 'Diff', 'RunType'])

    source = df[df['ID'].isin(big_change)].sort_values(by='Rank', ascending=False)
    order = list(source['ID'])
    print(len(source)/2, 'big changes')
    rank_changes = alt.Chart(source).mark_circle(size=100, filled=True, stroke='black',opacity=1).encode(
        x=alt.X('ID', sort=order,axis=alt.Axis(labelAngle=-60)),
        y=alt.Y('Rank', title='Exomiser Rank of Causative Variant'),#, scale=alt.Scale(domain=[0,29.1])),
        #size = 'Term_Diff',
        color=alt.Color('RunType',  scale=alt.Scale(scheme='category20', domain=domain), legend=alt.Legend(
        orient='top',
        direction='horizontal',
        titleAnchor='middle')),
        tooltip = ['RunType', 'ID', 'Rank', 'Diff']
    ).properties(
        height=250,
        width=150
    )

    source = df[df['ID'].isin(to_plot)].sort_values(by=['Diff', 'Rank'], ascending=[False,True])
    order = list(source[source['RunType']==str(filtered_run_type)].sort_values(by='Rank', ascending=False)['ID'])
    print(len(source)/2, 'changes')
    rank_changes_big = alt.Chart(source).mark_circle(size=100, filled=True, stroke='black',opacity=1).encode(
        x=alt.X('ID', sort=order,axis=alt.Axis(labelAngle=-60)),
        y=alt.Y('Rank', title='Exomiser Rank of Causative Variant'),#, scale=alt.Scale(domain=[0,29.1])),
        #size = 'Term_Diff',
        color=alt.Color('RunType',  scale=alt.Scale(scheme='category20', domain=domain), legend=alt.Legend(
        orient='top',
        direction='horizontal',
        titleAnchor='middle')),
        tooltip = ['RunType', 'ID', 'Rank', 'Diff']
    ).properties(
        height=250,
        width=150
    )


    source = df[df['ID'].isin(lost)].sort_values(by=['Diff', 'Rank'], ascending=[False,True])
    order = list(source[source['RunType']==str(unfiltered_run_type)].sort_values(by='Rank')['ID'])
    print(len(source)/2, 'lost')

    list(source['ID'])
    lost_plot = alt.Chart(source).mark_circle(size=100, filled=True, stroke='black',opacity=1).encode(
        x=alt.X('ID', sort=order,axis=alt.Axis(labelAngle=-60)),
        y=alt.Y('Rank', title='Exomiser Rank of Causative Variant'),#, scale=alt.Scale(domain=[0,29.1])),
        #size = 'Term_Diff',
        color=alt.Color('RunType',  scale=alt.Scale(scheme='category20', domain=domain), legend=alt.Legend(
        orient='top',
        direction='horizontal',
        titleAnchor='middle')),
        tooltip = ['RunType', 'ID', 'Rank', 'Diff']
    ).properties(
        height=250,
        width=150
    )



    source = df[df['ID'].isin(too_big)].sort_values(by=['Diff', 'Rank'], ascending=[False,True])
    order = list(source[source['RunType']==str(unfiltered_run_type)].sort_values(by='Rank')['ID'])
    print(len(source)/2, 'too big')

    list(source['ID'])
    too_big_plot = alt.Chart(source).mark_circle(size=100, filled=True, stroke='black',opacity=1).encode(
        x=alt.X('ID', sort=order,axis=alt.Axis(labelAngle=-60)),
        y=alt.Y('Rank', title='Exomiser Rank of Causative Variant'),#, scale=alt.Scale(domain=[0,29.1])),
        #size = 'Term_Diff',
        color=alt.Color('RunType',  scale=alt.Scale(scheme='category20', domain=domain), legend=alt.Legend(
        orient='top',
        direction='horizontal',
        titleAnchor='middle')),
        tooltip = ['RunType', 'ID', 'Rank', 'Diff']
    ).properties(
        height=250,
        width=150
    )

    source = df[df['ID'].isin(plot3)].sort_values(by='Diff', ascending=False)
    order = list(source['ID'])
    print(len(source)/2)
    

    return df, to_plot, big_change,rank_changes_big, rank_changes, lost_plot, too_big_plot



In [7]:
data = []
to_plot = []
success_table=success_table.fillna(0)
special_domain = ['noN_filtered_15_85_all_models_REMM_revel_mvp_PSF0_501_noWL_genomiser','b' , 'c','d','e','f','g','h','noN_filtered_15_85_human_REMM_revel_mvp_PSF0_501_noWL_genomiser'  ]

filtered_run_type = 'noN_filtered_15_85_human_REMM_revel_mvp_PSF0_501_noWL_genomiser'
unfiltered_run_type = 'noN_filtered_15_85_all_models_REMM_revel_mvp_PSF0_501_noWL_genomiser'
df, to_plot, big_change,rank_changes, rank_changes_big, lost_plot, too_big_plot = filt_vs_unfilt_df(unfiltered_run_type,filtered_run_type,success_table,  55, special_domain)


14.0 big changes
8.0 changes
6.0 lost
3.0 too big
23.0


In [8]:
genomiser_rank_changes = alt.hconcat(too_big_plot.properties(height=300,width=100)
            ,rank_changes_big.properties(height=300,width=250), 
                                      rank_changes.properties(height=300,width=150), 
                                      lost_plot.properties(height=300,width=100)).configure_axis(
    labelPadding= 5,
    labelLimit=0,
    labelFontSize=15, 
    titleFontSize=15, labelFont='arial', tickSize=8).configure_legend(
        labelLimit=0,labelFontSize=15, titleFontSize=15, labelFont='arial')

genomiser_rank_changes

alt.HConcatChart(...)